In [1]:
!pip install selenium

In [30]:
from selenium import webdriver
from selenium.webdriver import Chrome
import time 
import sqlite3
import os
from pandas.io import sql
import pandas as pd

In [31]:
def db_save(ARTICLE_LIST):
    with sqlite3.connect(os.path.join('.','data_go_kr_DB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            ARTICLE_LIST.to_sql(name = 'ARTICLE_LIST', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(ARTICLE_LIST), '건 저장완료..')

In [73]:
# 셀레늄의 웹 드라이버로 크롬 열기
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

browser = webdriver.Chrome('chromedriver', options = options)


# 원하는 페이지로 이동
browser.get('http://gbiga.onilifo.co.kr/')
browser.implicitly_wait(3)

# 로그인 창 열기
browser.find_element_by_xpath('/html/body/div[1]/div[1]/div[2]/ul/li[6]/ul/li/a').click()
browser.implicitly_wait(3)

# 로그인 하기
import json
with open('account.json', 'r') as f:
    json_data = json.load(f)
browser.find_element_by_xpath('//*[@id="username"]').send_keys(json_data['pid'])
browser.find_element_by_xpath('//*[@id="password"]').send_keys(json_data['pwd'])
browser.find_element_by_xpath('//*[@id="btn-login"]').click()
browser.implicitly_wait(3)

# 작업 수행하기 (수강신청 목록 보기)
browser.find_element_by_xpath('//*[@id="link-registration"]').click()
browser.implicitly_wait(3)

sect1 = browser.find_elements_by_xpath('/html/body/div[1]/div[3]/div/div[3]/ul/li')
df_list1 = []
for s in sect1:
    df = pd.DataFrame({
        'TITLE' : [s.find_element_by_xpath('div/div/div[2]/h1').text],
        'TIME' : [s.find_element_by_xpath('div/div/div[2]/div/p').text]
    })
    df_list1.append(df)
sub_list1 = pd.concat(df_list1)
db_save(sub_list1)

sect2 = browser.find_elements_by_xpath('/html/body/div[1]/div[3]/div/div')
df_list2 = []
for s in sect2:
    try: # 가장 힘들었던 부분 : 알고보니 마지막 div에는 NAME에 해당하는 제목이 없어서 문제가 발생 -> try ~ except 구문으로 처리..
        df = pd.DataFrame({
            'NAME' : [s.find_element_by_xpath('div[1]/h1').text]
        })
        df_list2.append(df)
    except:
        pass
sub_list2 = pd.concat(df_list2)
db_save(sub_list2)

table ARTICLE_LIST has no column named TIME
3 건 저장완료..
table ARTICLE_LIST has no column named NAME
3 건 저장완료..


In [74]:
sub_list1

,TITLE,TIME
0,[07.06(화)] 데이터에 대한 이해와 정형데이터 다루기(모델링)_A,1시간 46분
0,[07.16(금)] SQL을 이용한 정형데이터 다루기(SQL기초)_A,2시간 37분
0,[07.08(목)] R로 배우는 데이터분석 이해(통계기초이론)_A,1시간 22분


In [75]:
sub_list2

,NAME
0,공공빅데이터 Overview
0,공공 빅데이터 리허설
0,분석가를 위한 데이터역량
